## Connecting with a MongoDB on the cloud using mLab

In [74]:
import pymongo
try:
    #use your database name, user and password here:
    #mongodb://<dbuser>:<dbpassword>@<mlab_url>.mlab.com:57066/<database_name>
    with open("credentials", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
    print([name,password,url,dbname])
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
print(conn)

#Create a database using db = conn.name_db or dictionary access db = conn['name_db']
db = conn[dbname]
print (db)
collection = db.tweets
db.collection_names()

Connected successfully!!!
['alex', 'alex', 'ds145293.mlab.com:45293', 'alex']
MongoClient(host=['ds145293.mlab.com:45293'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['ds145293.mlab.com:45293'], document_class=dict, tz_aware=False, connect=True), 'alex')


['objectlabs-system', 'system.indexes', 'objectlabs-system.admin.collections']

In [75]:
import pandas as pd
df = pd.read_json('./TweetsBarcelona/test.json',orient='index')
df

,createdAt,geoLocation,id,inReplyToUserId,text,userId,userLang,userLocation
tweet1,30-mar-2014 23:51:46,NaN,450389948961943552,728530951,@Weedshine Aquesta pinta millor. O pots fer un...,246452645,ca,"Raval, Pa?sos Catalans"
tweet2,30-mar-2014 23:51:46,"{'latitude': 41.5292058, 'longitude': 1.8507183}",450389949700141056,-1,Jaja = ?,595619618,ca,Catalunya.
tweet3,30-mar-2014 23:51:38,"{'latitude': 41.37657536, 'longitude': 2.16752...",450389916661592064,-1,CADA UNO DA LO QUE TIENE http://t.co/xjSJw7CJf7,248747003,es,Estepa (Sevilla) Espa?a


In [87]:
collection.insert_many(df.to_dict("records"))

In [77]:
#conn.close()

In [96]:
#db.drop_collection("tweets")

{'nIndexesWas': 1, 'ns': 'alex.tweets', 'ok': 1.0}

## Twitter api usage

In [79]:
import json
import pymongo
import tweepy

with open('consumer_key', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret', 'r') as f:
     access_secret = f.read()
f.closed


#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#Do something
USER_NAME = "usuarialatzar"
user = api.get_user(id=USER_NAME)
print (user.name)

PSKS


In [102]:
from tweepy import Stream,StreamListener

class listener(StreamListener):
    def __init__(self):
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        self.db = db
        self.collection = self.db.tweets
    
    def on_data(self, status):
        if self.num_tweets < 5:
            jdata = json.loads(status)
            if jdata['coordinates'] is not None or jdata['geo'] is not None:
                document={'geo':jdata['geo'],'coordinates':jdata['coordinates'],'text':jdata["text"], 'created':jdata["created_at"]}
                self.collection.insert_one(document) 
                print (jdata['geo'],jdata["text"])
                self.num_tweets += 1
            return True
        else:
            return False
    
    def on_error(self, status):
        print (status)

# Catch all tweets in Barcelona area and print them
twitterStream = Stream(auth, listener()) 
twitterStream.filter(locations=[2.0504377635,41.2787636541,2.3045074059,41.4725622346])
print ("done")

{'type': 'Point', 'coordinates': [41.38388889, 2.17638889]} #exposiciodart #nofilter @ Barcelona Cathedral https://t.co/yqDHca54Rh
{'type': 'Point', 'coordinates': [41.37744629, 2.16472257]} 4 self-portraits (in reverse order of execution) #penandinkart #selfportrait @ Poble Sec / Montjuic https://t.co/4vfpENa7ah


KeyboardInterrupt: 

In [98]:
collection.count()
print (collection.count())
for doc in collection.find():
    print (doc)

5
{'_id': ObjectId('59fa71ecec130e3bac2e1f40'), 'id': 925894339108261888, 'place': {'id': '1a27537478dd8e38', 'url': 'https://api.twitter.com/1.1/geo/id/1a27537478dd8e38.json', 'place_type': 'city', 'name': 'Barcelona', 'full_name': 'Barcelona, Spain', 'country_code': 'ES', 'country': 'Spain', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[2.052477, 41.317048], [2.052477, 41.468266], [2.229978, 41.468266], [2.229978, 41.317048]]]}, 'attributes': {}}, 'coordinates': {'type': 'Point', 'coordinates': [2.16992, 41.3879]}, 'text': 'Alba Sánchez, @albasanchz es ahora una tendencia en #Barcelona\n\nhttps://t.co/Li62Bkv7WA https://t.co/kNXZtwI7Rz', 'created': 'Thu Nov 02 01:16:27 +0000 2017'}
{'_id': ObjectId('59fa7200ec130e3bac2e1f41'), 'id': 925894422914654208, 'place': {'id': '1a27537478dd8e38', 'url': 'https://api.twitter.com/1.1/geo/id/1a27537478dd8e38.json', 'place_type': 'city', 'name': 'Barcelona', 'full_name': 'Barcelona, Spain', 'country_code': 'ES', 'country': 'Spain', 'bound